# MBA Investment Management – Performance Evaluation Demo

In this notebook, we will walk through:
1. Downloading stock data and Fama-French factor data
2. Constructing a simple portfolio
3. Running regressions to get CAPM and multi-factor alphas
4. Calculating risk-adjusted performance metrics (Sharpe, Information, Treynor)

This follows from the lecture slides on performance evaluation and uses real market data.


## 1. Imports and Helper Functions
We will need a few common libraries: `yfinance`, `pandas_datareader`, `pandas`, `numpy`, `matplotlib`, `statsmodels`.

In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt
from pandas_datareader import data as web
import statsmodels.api as sm

# Make plots appear inline (in a Jupyter environment)
%matplotlib inline

def get_stock_data(tickers, start_date, end_date):
    """
    Download historical stock data (Close prices) for a list of tickers.
    Returns daily returns.
    """
    data = yf.download(tickers, start=start_date, end=end_date)["Close"]
    returns = data.pct_change().dropna()
    return returns


ModuleNotFoundError: No module named 'pandas_datareader'

## 2. Data Preparation

### 2.1 Download Stock and Market Data
Let's get 5 years of daily data for several stocks plus the S&P 500 index (to use as our market portfolio).

In [ ]:
# Define date range - ~5 years of data
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=5*365)

# Define tickers - some popular stocks
market_ticker = '^GSPC'  # S&P 500
stock_tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'NVDA', 'JPM', 'V', 'WMT']

# Combined list with market first (useful for certain calculations)
tickers = [market_ticker] + stock_tickers

# Download the data
returns = get_stock_data(tickers, start_date, end_date)

returns.head()

### 2.2 Calculate Excess Returns
We assume a constant risk-free rate (annualized 3%) and convert it to a daily rate for simplicity. Then we subtract it from the raw returns to get excess returns.

In [ ]:
risk_free_rate_annual = 0.03  # 3% annual
risk_free_rate_daily = risk_free_rate_annual / 252  # approximate daily risk-free

excess_returns = returns.subtract(risk_free_rate_daily, axis=0)
excess_returns.head()

### 2.3 Fama-French Data
We'll grab the Fama-French 3-Factor data from Kenneth French’s library using `pandas_datareader`. Note that 'F-F_Research_Data_Factors' is provided in monthly frequency. If you want daily factor data, you need a different dataset from the library. For this example, let's proceed with the monthly data. We'll also convert the daily stock returns to monthly to match.


In [ ]:
# Get monthly Fama-French 3-factor data (Mkt-RF, SMB, HML, RF)
ff_factors = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start='1950')[0]
# Convert to decimals
ff_factors = ff_factors / 100
ff_factors.head()

### 2.4 Aligning Frequencies (Converting to Monthly)
Because our stock returns are currently daily, let's convert them to monthly (taking the arithmetic sum of daily returns for the month is usually done by `(1 + daily_returns).prod() - 1` for total return over the month). Then we can easily join them with the Fama-French monthly data.


In [ ]:
# Convert daily returns to monthly returns
monthly_returns = (1 + returns).resample('M').prod() - 1
monthly_excess_returns = monthly_returns.subtract(risk_free_rate_annual/12, axis=1)

# We'll rename columns so that the market ticker is 'Market'
rename_map = {market_ticker: 'Market'}
monthly_excess_returns = monthly_excess_returns.rename(columns=rename_map)

# Join with Fama-French
# We'll shift the Fama-French index so it aligns by last day of the month.
# Alternatively, we can leave as is. Just ensure consistent alignment in practice.

ff_data = ff_factors.copy()
# Make sure to convert index to end-of-month for alignment
ff_data.index = ff_data.index.to_period('M').to_timestamp('M')
combined_data = pd.merge(monthly_excess_returns, ff_data, left_index=True, right_index=True, how='inner')

combined_data.head()

## 3. Construct a Portfolio
For simplicity, let's assume an equally weighted portfolio of our 10 stocks (excluding the market index). We'll compute its monthly returns and then proceed to measure performance.


In [ ]:
# List of actual stocks (exclude 'Market' which was ^GSPC)
portfolio_stocks = [t for t in monthly_excess_returns.columns if t != 'Market']

# Equally weighted portfolio
weights = np.array([1/len(portfolio_stocks)]*len(portfolio_stocks))

# Calculate portfolio returns (just do a dot product across columns)
portfolio_returns = monthly_excess_returns[portfolio_stocks].mul(weights).sum(axis=1)
portfolio_returns.name = 'Portfolio'

portfolio_returns.head()

## 4. Factor Regressions and Alpha
We'll run a CAPM regression, then a 3-factor (Fama-French) regression, on our portfolio.

### 4.1 CAPM Regression
CAPM: 
\[
r_{p,t} - r_{f,t} = \alpha + \beta (r_{m,t} - r_{f,t}) + \epsilon_t
\]

In the Fama-French data, `Mkt-RF` is already `(Market - RF)`. If we want to do a basic CAPM in that style, we can just do:
\[
r_{p,t} - r_{f,t} = \alpha + \beta (\text{Mkt-RF}) + \epsilon_t
\]
But note that our `portfolio_returns` are already *excess returns*.

In [ ]:
# Merge portfolio returns with factors
reg_data = pd.concat([
    portfolio_returns,
    ff_data['Mkt-RF']
], axis=1).dropna()

Y = reg_data['Portfolio']  # portfolio excess return
X = reg_data['Mkt-RF']
X = sm.add_constant(X)

capm_model = sm.OLS(Y, X).fit()
print(capm_model.summary())

### 4.2 Three-Factor Regression
Now we extend to the Fama-French 3-factor model:
\[
r_{p,t} - r_{f,t} = \alpha + \beta (\text{Mkt-RF}) + s\,\text{SMB} + h\,\text{HML} + \epsilon_t
\]


In [ ]:
# Merge the portfolio returns with Mkt-RF, SMB, HML
ff_3factor_data = pd.concat([
    portfolio_returns,
    ff_data[['Mkt-RF','SMB','HML']]
], axis=1).dropna()

Y = ff_3factor_data['Portfolio']
X = ff_3factor_data[['Mkt-RF','SMB','HML']]
X = sm.add_constant(X)

ff3_model = sm.OLS(Y, X).fit()
print(ff3_model.summary())

## 5. Risk-Adjusted Performance Measures
We’ll compute:
- **Sharpe Ratio**
- **Information Ratio** (relative to CAPM)
- **Treynor Ratio**

### 5.1 Sharpe Ratio
Recall:
\[ \text{Sharpe Ratio} = \frac{\bar{r}_p - r_f}{\sigma_p} \]
Since we’ve already been working in excess returns, we can just do:
\[ \text{Sharpe Ratio} = \frac{\bar{r}_p}{\sigma_p} \]
where \( \bar{r}_p\) is the average excess return on the portfolio (monthly if that’s how your data is). We'll then annualize that if needed.


In [ ]:
# Monthly average (excess) return of portfolio
mean_rp_monthly = portfolio_returns.mean()
# Monthly std dev
std_rp_monthly = portfolio_returns.std()

sharpe_monthly = mean_rp_monthly / std_rp_monthly

# If you want an annualized Sharpe ratio for monthly data:
# annualized_sharpe = sharpe_monthly * np.sqrt(12)

print("Monthly Sharpe Ratio = ", round(sharpe_monthly, 4))
print("Annualized Sharpe Ratio = ", round(sharpe_monthly * np.sqrt(12), 4))

### 5.2 Information Ratio
The Information Ratio is defined as:
\[ IR = \frac{\alpha_p}{\sigma(\epsilon_p)} \]
where \( \alpha_p \) is the portfolio’s alpha (e.g., from CAPM) and \( \sigma(\epsilon_p) \) is the standard deviation of the residual.

From our CAPM regression, we can extract alpha and the residuals (i.e., fitted errors).

In [ ]:
# From our CAPM regression:
alpha_capm = capm_model.params["const"]
residuals = capm_model.resid
sigma_e = residuals.std()
# From our FF3 regression:
alpha_ff3 = ff3_model.params["const"]
residuals_ff3 = ff3_model.resid
sigma_e_ff3 = residuals_ff3.std()

# IR in monthly terms
IR_monthly = alpha_capm / sigma_e
IR_monthly_ff3 = alpha_ff3 / sigma_e_ff3


# If you want to interpret alpha as an annual figure, you might multiply alpha by 12
# but should also scale the std dev consistently. We'll just keep everything in monthly.

print("CAPM")
print("CAPM alpha (monthly) =", round(alpha_capm, 5))
print("Residual stdev (monthly) =", round(sigma_e, 5))
print("Information Ratio (monthly) =", round(IR_monthly, 4))

print("FF3")
print("FF3 alpha (monthly) =", round(alpha_capm, 5))
print("Residual stdev (monthly) =", round(sigma_e, 5))
print("Information Ratio (monthly) =", round(IR_monthly, 4))

### 5.3 Treynor Ratio
The Treynor ratio uses *beta* rather than total volatility:
\[ T = \frac{\bar{r}_p - r_f}{\beta_p}. \]
Again, we’re working in excess returns for the portfolio, so it is simply:
\[ T = \frac{\bar{r}_p}{\beta_p}. \]

In [ ]:
# CAPM beta is in capm_model.params["Mkt-RF"]
beta_capm = capm_model.params["Mkt-RF"]

treynor_monthly = mean_rp_monthly / beta_capm

# If needed, you could annualize the numerator. We'll just keep monthly.

print("CAPM Beta =", round(beta_capm, 4))
print("Treynor Ratio (monthly) =", round(treynor_monthly, 5))

# 6. Brimson's Decomposition

In [ ]:



def brinson_decomposition(df):
    """
    Given a DataFrame df with columns:
        'w_p': portfolio weight in each asset class
        'r_p': portfolio return in each asset class
        'w_b': benchmark weight in each asset class
        'r_b': benchmark return in each asset class
    this function computes and returns the Brinson decomposition:
      1) Asset Allocation
      2) Selection
      3) Interaction
    plus the total difference in returns.
    """
    # Total portfolio and benchmark returns
    r_p = (df['w_p'] * df['r_p']).sum()
    r_b = (df['w_b'] * df['r_b']).sum()

    # Decomposition components
    asset_allocation = ((df['w_p'] - df['w_b']) * df['r_b']).sum()
    selection = (df['w_b'] * (df['r_p'] - df['r_b'])).sum()
    interaction = ((df['w_p'] - df['w_b']) * (df['r_p'] - df['r_b'])).sum()

    # Total difference in returns
    total_diff = r_p - r_b

    return {
        'Portfolio Return': r_p,
        'Benchmark Return': r_b,
        'Asset Allocation': asset_allocation,
        'Selection': selection,
        'Interaction': interaction,
        'Total Difference': total_diff
    }


# EXAMPLE USAGE:
# -------------------------------------------------
# Imagine we have a small universe of 3 “asset classes” (or sectors).
# Create a DataFrame with columns for portfolio weights/returns
# and benchmark weights/returns:

data = {
    'AssetClass':    ['Equity', 'Bond', 'RealEstate'],
    'w_p':           [0.60,      0.30,   0.10],   # Portfolio weights
    'r_p':           [0.10,      0.02,   0.08],   # Portfolio returns
    'w_b':           [0.50,      0.40,   0.10],   # Benchmark weights
    'r_b':           [0.08,      0.03,   0.07]    # Benchmark returns
}

df_brinson = pd.DataFrame(data).set_index('AssetClass')

# Perform Brinson decomposition
results = brinson_decomposition(df_brinson)

print("Brinson Decomposition Results:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")

NameError: name 'pd' is not defined

### Conclusion
In this notebook, we've shown how to:
- Pull real data from Yahoo Finance and Fama-French,
- Construct an equal-weight portfolio,
- Run factor regressions (CAPM, Fama-French 3-factor),
- Compute common risk-adjusted performance metrics (Sharpe, Information, Treynor).

This provides a foundation for further exploration and experimentation in an MBA-level investment management course.